In [ ]:
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import json
import os

In [ ]:
env_path = find_dotenv(raise_error_if_not_found=True)

## Create an AKS cluster

In [ ]:
aks_cluster = "<your-aks-cluster-name>"
set_key(env_path, "AKS_CLUSTER", aks_cluster)

In [ ]:
!az aks create \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --name {get_key(env_path, "AKS_CLUSTER")} \
    --node-count 3 \
    --node-vm-size "Standard_NC6s_v2"

## Install kubectl

In [ ]:
!sudo az aks install-cli

In [ ]:
!az aks get-credentials \
    --resource-group {get_key(env_path, 'RESOURCE_GROUP')}\
    --name {get_key(env_path, 'AKS_CLUSTER')}

In [ ]:
!kubectl get nodes

In [ ]:
!kubectl get pods --all-namespaces

## Add items to queue

In [ ]:
!python queuer_app/add_images_to_queue.py --blob-dir "input" --queue-limit 9

In [ ]:
!az servicebus queue show \
    --name {get_key(env_path, "SB_QUEUE")} \
    --namespace-name {get_key(env_path, "SB_NAMESPACE")} \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --query "countDetails.activeMessageCount"

## Deploy your dequeuer app to the AKS cluster

In [ ]:
dequeuer_json = {
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "dequeuer", 
        "labels": {
            "purpose": "dequeue_messages"
        }
    },
    "spec": {
        "replicas": 3,
        "template": {
            "metadata": {
                "labels": {
                    "app": "dequeuer"
                }
            },
            "spec": {
                "containers": [
                    {
                        "name": "dequeuer",
                        "image": "{}/{}:latest".format(get_key(env_path, "DOCKER_LOGIN"), get_key(env_path, "DEQUEUER_IMAGE_REPO")),
                        "volumeMounts": [
                            {
                                "mountPath": "/usr/local/nvidia", 
                                "name": "nvidia"
                            }
                        ],
                        "resources": {
                            "requests": {
                                "alpha.kubernetes.io/nvidia-gpu": 1
                            },
                            "limits": {
                                "alpha.kubernetes.io/nvidia-gpu": 1
                            },
                        },
                        "ports": [{
                            "containerPort": 433
                        }],
                        "env": [
                            {
                                "name": "LB_LIBRARY_PATH",
                                "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.6/lib",
                            },
                            {
                                "name": "DP_DISABLE_HEALTHCHECKS", 
                                "value": "xids"
                            },
                            {
                                "name": "STYLE_WEIGHT",
                                "value": get_key(env_path, "STYLE_WEIGHT")
                            },
                            {
                                "name": "CONTENT_WEIGHT",
                                "value": get_key(env_path, "CONTENT_WEIGHT")
                            },
                            {
                                "name": "NUM_STEPS",
                                "value": get_key(env_path, "NUM_STEPS")
                            },
                            {
                                "name": "IMAGE_SIZE",
                                "value": get_key(env_path, "IMAGE_SIZE")
                            },
                            {
                                "name": "VIDEO_NAME",
                                "value": get_key(env_path, "VIDEO_NAME")
                            },
                            {
                                "name": "SUBSCRIPTION_ID",
                                "value": get_key(env_path, "SUBSCRIPTION_ID")
                            },
                            {
                                "name": "RESOURCE_GROUP",
                                "value": get_key(env_path, "RESOURCE_GROUP")
                            },
                            {
                                "name": "REGION",
                                "value": get_key(env_path, "REGION")
                            },
                            {
                                "name": "STORAGE_ACCOUNT_NAME", 
                                "value": get_key(env_path, "STORAGE_ACCOUNT_NAME")
                            },
                            {
                                "name": "STORAGE_ACCOUNT_KEY",
                                "value": get_key(env_path, "STORAGE_ACCOUNT_KEY")
                            },
                            {
                                "name": "STORAGE_CONTAINER_NAME",
                                "value": get_key(env_path, "STORAGE_CONTAINER_NAME")
                            },
                            {
                                "name": "SB_SHARED_ACCESS_KEY_NAME",
                                "value": get_key(env_path, "SB_SHARED_ACCESS_KEY_NAME")
                            },
                            {
                                "name": "SB_SHARED_ACCESS_KEY_VALUE",
                                "value": get_key(env_path, "SB_SHARED_ACCESS_KEY_VALUE")
                            },
                            {
                                "name": "SB_NAMESPACE",
                                "value": get_key(env_path, "SB_NAMESPACE")
                            },
                            {
                                "name": "SB_QUEUE", 
                                "value": get_key(env_path, "SB_QUEUE")
                            },
                        ],
                    }
                ],
                "volumes": [
                    {
                        "name": "nvidia", 
                        "hostPath": {
                            "path": "/usr/local/nvidia"
                        }
                    }
                ],
            },
        },
    },
}

In [ ]:
with open("dequeuer_app.json", "w") as outfile:
    json.dump(dequeuer_json, outfile, indent=4, sort_keys=True)
    outfile.write('\n\n')

In [ ]:
!kubectl create -f dequeuer_app.json

In [ ]:
!kubectl get pods --all-namespaces

In [ ]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

---

## Conclusion

In [ ]:
!cat .env